In [ ]:
import jsonlines
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import lightgbm as lgb
import warnings
import os

DATA_PATH = os.path.dirname(os.getcwd())

In [28]:
def get_diplomacy_map():
    """
    Returns a dictionary where each key is a territory and the value
    is a list of territories it borders.
    This is the static map of the game.
    """
    # This map defines all territory adjacencies in Diplomacy
    return {
        'Cly': ['Edi', 'Lvp', 'Nao', 'Nwg'],
        'Edi': ['Cly', 'Lvp', 'Nth', 'Nwg', 'Yor'],
        'Lvp': ['Cly', 'Edi', 'Iri', 'Nao', 'Wal', 'Yor'],
        'Yor': ['Edi', 'Lvp', 'Lon', 'Nth', 'Wal'],
        'Wal': ['Lvp', 'Yor', 'Lon', 'Eng', 'Iri'],
        'Lon': ['Yor', 'Wal', 'Eng', 'Nth'],
        'Nwy': ['Bar', 'Nwg', 'Nth', 'Ska', 'Stp', 'Swe'],
        'Swe': ['Nwy', 'Ska', 'Den', 'Bal', 'Bot', 'Fin'],
        'Den': ['Swe', 'Ska', 'Nth', 'Hel', 'Kie', 'Bal'],
        'Kie': ['Den', 'Hel', 'Hol', 'Ruh', 'Mun', 'Ber', 'Bal'],
        'Ber': ['Kie', 'Bal', 'Pru', 'Sil', 'Mun'],
        'Pru': ['Ber', 'Bal', 'Lvn', 'War', 'Sil'],
        'Stp': ['Nwy', 'Bar', 'Mos', 'Lvn', 'Bot', 'Fin'],
        'Fin': ['Nwy', 'Stp', 'Bot', 'Swe'],
        'Lvn': ['Stp', 'Mos', 'War', 'Pru', 'Bal', 'Bot'],
        'Mos': ['Stp', 'Lvn', 'War', 'Ukr', 'Sev'],
        'War': ['Pru', 'Lvn', 'Mos', 'Ukr', 'Gal', 'Sil'],
        'Sil': ['Ber', 'Mun', 'Boh', 'Gal', 'War', 'Pru'],
        'Hol': ['Kie', 'Hel', 'Nth', 'Eng', 'Bel', 'Ruh'],
        'Bel': ['Hol', 'Eng', 'Pic', 'Bur', 'Ruh'],
        'Ruh': ['Kie', 'Hol', 'Bel', 'Bur', 'Mun'],
        'Mun': ['Kie', 'Ruh', 'Bur', 'Tyr', 'Boh', 'Sil', 'Ber'],
        'Boh': ['Mun', 'Tyr', 'Vie', 'Gal', 'Sil'],
        'Gal': ['Boh', 'Vie', 'Bud', 'Rum', 'Ukr', 'War', 'Sil'],
        'Ukr': ['Mos', 'War', 'Gal', 'Rum', 'Sev'],
        'Sev': ['Mos', 'Ukr', 'Rum', 'Bla', 'Arm'],
        'Bre': ['Eng', 'Mao', 'Pic', 'Par', 'Gas'],
        'Pic': ['Eng', 'Bel', 'Bur', 'Par', 'Bre'],
        'Par': ['Pic', 'Bur', 'Gas', 'Bre'],
        'Bur': ['Bel', 'Ruh', 'Mun', 'Mar', 'Gas', 'Par', 'Pic'],
        'Gas': ['Bre', 'Par', 'Bur', 'Mar', 'Spa', 'Mao'],
        'Mar': ['Bur', 'Gas', 'Spa', 'Lyo'],
        'Spa': ['Gas', 'Mar', 'Lyo', 'Wes', 'Mao', 'Por'],
        'Por': ['Spa', 'Mao'],
        'Nao': ['Cly', 'Lvp', 'Iri', 'Mao'],
        'Iri': ['Lvp', 'Wal', 'Eng', 'Mao', 'Nao'],
        'Eng': ['Wal', 'Lon', 'Nth', 'Hol', 'Bel', 'Pic', 'Bre', 'Mao', 'Iri'],
        'Mao': ['Bre', 'Gas', 'Spa', 'Por', 'Nao', 'Iri', 'Eng', 'Wes'],
        'Nth': ['Edi', 'Yor', 'Lon', 'Eng', 'Hol', 'Hel', 'Den', 'Ska', 'Nwy', 'Nwg'],
        'Nwg': ['Cly', 'Edi', 'Nth', 'Nwy', 'Bar'],
        'Bar': ['Nwy', 'Nwg', 'Stp'],
        'Hel': ['Den', 'Kie', 'Hol', 'Nth'],
        'Ska': ['Nwy', 'Swe', 'Den', 'Nth'],
        'Bal': ['Swe', 'Den', 'Kie', 'Ber', 'Pru', 'Lvn', 'Bot'],
        'Bot': ['Swe', 'Fin', 'Stp', 'Lvn', 'Bal'],
        'Wes': ['Spa', 'Mar', 'Lyo', 'Tyn', 'Mao'],
        'Lyo': ['Mar', 'Spa', 'Wes', 'Tyn', 'Pie', 'Bur'],
        'Tyn': ['Wes', 'Lyo', 'Ion', 'Tun', 'Rom'],
        'Ion': ['Tyn', 'Rom', 'Nap', 'Apu', 'Adr', 'Alb', 'Gre', 'Aeg', 'Eas'],
        'Adr': ['Ven', 'Tri', 'Alb', 'Ion', 'Apu'],
        'Aeg': ['Gre', 'Ion', 'Eas', 'Smy', 'Con', 'Bul'],
        'Eas': ['Ion', 'Aeg', 'Smy', 'Syr'],
        'Bla': ['Sev', 'Arm', 'Ank', 'Con', 'Bul', 'Rum'],
        'Pie': ['Mar', 'Lyo', 'Tyr', 'Ven'],
        'Ven': ['Pie', 'Tyr', 'Tri', 'Adr', 'Apu', 'Rom'],
        'Rom': ['Ven', 'Apu', 'Nap', 'Tyn', 'Lyo', 'Pie'],
        'Nap': ['Rom', 'Apu', 'Ion'],
        'Apu': ['Ven', 'Adr', 'Ion', 'Nap', 'Rom'],
        'Tyr': ['Mun', 'Boh', 'Vie', 'Tri', 'Ven', 'Pie', 'Bur'],
        'Vie': ['Boh', 'Gal', 'Bud', 'Tri', 'Tyr'],
        'Tri': ['Vie', 'Bud', 'Ser', 'Alb', 'Adr', 'Ven', 'Tyr'],
        'Bud': ['Vie', 'Gal', 'Rum', 'Ser', 'Tri'],
        'Rum': ['Gal', 'Ukr', 'Sev', 'Bla', 'Bul', 'Ser', 'Bud'],
        'Ser': ['Bud', 'Rum', 'Bul', 'Gre', 'Alb', 'Tri'],
        'Alb': ['Tri', 'Ser', 'Gre', 'Ion', 'Adr'],
        'Gre': ['Ser', 'Alb', 'Ion', 'Aeg', 'Bul'],
        'Bul': ['Rum', 'Bla', 'Con', 'Aeg', 'Gre', 'Ser'],
        'Con': ['Bul', 'Aeg', 'Smy', 'Ank', 'Bla'],
        'Ank': ['Con', 'Bla', 'Arm', 'Smy'],
        'Arm': ['Sev', 'Bla', 'Ank', 'Smy', 'Syr'],
        'Smy': ['Con', 'Ank', 'Arm', 'Syr', 'Eas', 'Aeg'],
        'Syr': ['Arm', 'Smy', 'Eas'],
        'Tun': ['Tyn', 'Ion', 'Naf'],
        'Naf': ['Tun']
    }

In [29]:
def are_players_adjacent(sender, receiver, territories_data, board_map):
    """
    Checks if a sender and receiver control any bordering territories.
    """
    sender_territories = set()
    receiver_territories = set()
    
    # 1. Find all territories for each player
    # territories_data is like: {"Par": "France", "Smy": "Turkey", ...}
    for territory, owner in territories_data.items():
        if owner.lower() == sender:
            sender_territories.add(territory)
        elif owner.lower() == receiver:
            receiver_territories.add(territory)
            
    # 2. Check for adjacency
    for sender_terr in sender_territories:
        # Check if this territory is even on the map (it might be a sea zone)
        if sender_terr in board_map:
            # Look up all neighbors
            for neighbor in board_map[sender_terr]:
                # If any neighbor is owned by the receiver, they are adjacent
                if neighbor in receiver_territories:
                    return True
                    
    return False

In [30]:
import json
import re
from collections import defaultdict

def create_game_state_dictionary(game_state_dir):
    """
    Scans a directory of game state JSON files and loads them into a
    nested dictionary structured as:
    game_states[game_id][year][season] = file_content
    """

    filename_pattern = re.compile(r"DiplomacyGame(\d+)_(\d+)_(\w+)\.json")

    # We use defaultdict to automatically create nested dictionaries
    game_states = defaultdict(lambda: defaultdict(dict))
    
    print(f"Starting to build game state dictionary from: {game_state_dir}")
    
    file_count = 0
    for filename in os.listdir(game_state_dir):
        match = filename_pattern.match(filename)
        
        if match:
            # 1. Parse filename
            game_id = int(match.group(1)) # Convert game ID to integer
            year = match.group(2)         # Keep year as a string (like "1901")
            season = match.group(3).lower()       # Keep season as a string (like "Spring")
            
            # 2. Load the JSON file content
            file_path = os.path.join(game_state_dir, filename)
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)
                    
                    # 3. Store it in the nested dictionary
                    game_states[game_id][year][season] = data
                    file_count += 1
            except Exception as e:
                print(f"Warning: Could not load or parse {filename}. Error: {e}")
        else:
            print(f"Warning: Skipping file with unxpected name format: {filename}")

    print(f"Successfully loaded and indexed {file_count} game state files.")
    return game_states

# --- How to use it ---

# 1. DEFINE the path to your folder containing the game state files
GAME_STATE_DIRECTORY = os.path.join(DATA_PATH, "data", "raw", "2020_acl_diplomacy", "moves") # <-- IMPORTANT: Change this to your folder path

# 2. RUN the function to build the dictionary
#    (This might take a moment if you have thousands of files)
game_state_lookup = create_game_state_dictionary(GAME_STATE_DIRECTORY)

# 3. TEST the dictionary (optional)
if 1 in game_state_lookup and '1901' in game_state_lookup[1] and 'Spring' in game_state_lookup[1]['1901']:
    print("\nTest lookup successful!")
    # You can inspect a single game state like this:
    # print(game_state_lookup[1]['1901']['Spring']['orders'])
else:
    print("\nTest lookup failed. Check your directory path and filename pattern.")

Starting to build game state dictionary from: c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\moves
Successfully loaded and indexed 342 game state files.

Test lookup failed. Check your directory path and filename pattern.


In [31]:
DIPLOMACY_MAP = get_diplomacy_map()

def extract_features_from_state(message_data, game_state):
    """
    Extracts features, now including player adjacency.
    """
    new_features = {}
    
    sender = message_data['speaker'].lower()
    receiver = message_data['receiver'].lower()
    
    # === 1. Extract SC Counts (Same as before) ===
    sc_text = game_state.get('sc', '')
    sc_pattern = re.compile(r"\s*(\w+)\s+(\d+)\s*")
    all_scores = {}
    for match in sc_pattern.finditer(sc_text):
        country = match.group(1).lower()
        score = int(match.group(2))
        all_scores[country] = score
    new_features['sender_sc'] = all_scores.get(sender, 0)
    new_features['receiver_sc'] = all_scores.get(receiver, 0)

    # === 2. Extract Unit Counts (Same as before) ===
    orders_data = game_state.get('orders', {})
    sender_units = orders_data.get(sender.upper(), {})
    receiver_units = orders_data.get(receiver.upper(), {})
    new_features['sender_unit_count'] = len(sender_units)
    new_features['receiver_unit_count'] = len(receiver_units)
    
    # === 3. Add Original Score Delta (Same as before) ===
    new_features['score_delta'] = message_data['score_delta']
    
    # === 4. NEW FEATURE: Adjacency ===
    territories_data = game_state.get('territories', {})
    
    is_adjacent = are_players_adjacent(sender, receiver, territories_data, DIPLOMACY_MAP)
    
    # Convert boolean True/False to 1/0 for the model
    new_features['are_adjacent'] = 1 if is_adjacent else 0
    
    return new_features


# --- This is our MODIFIED loading function ---
# Create a global set to store keys we've already warned about
FAILED_KEYS = set()

def load_and_flatten_data(filepath, game_state_lookup):
    """
    Loads the .jsonl file, uses the game_state_lookup to find
    the matching game state, and extracts features.
    """
    X_text_list = []
    y_list = []
    X_new_features_list = [] 

    with jsonlines.open(filepath, 'r') as reader:
        for game in reader:
            game_id = game['game_id']
            years = game['years']
            seasons = game['seasons']
            speakers = game['speakers']
            
            for i in range(len(game['messages'])):
                sender_label = game['sender_labels'][i]
                
                if sender_label == "NOANNOTATION":
                    continue
                
                label = 1 if sender_label == False else 0 # Lie = 1, True = 0
                message_text = game['messages'][i]
                year = years[i]
                season = seasons[i].lower()
                
                try:
                    current_game_state = game_state_lookup[game_id][year][season]
                
                except KeyError:
                    # --- THIS IS THE NEW DEBUG CODE ---
                    # Create a unique key for this message
                    key = f"GameID: {game_id}, Year: {year}, Season: {season}"
                    
                    # Only print the warning once per unique key
                    if key not in FAILED_KEYS:
                        print(f"  [Debug Warning] Failed to find state for: {key}")
                        FAILED_KEYS.add(key)
                    # --- END DEBUG CODE ---
                    
                    continue # Skip this message
                
                message_data = {
                    'text': message_text,
                    'speaker': speakers[i],
                    'receiver': game['receivers'][i],
                    'score_delta': int(game['game_score_delta'][i])
                }
                
                new_features = extract_features_from_state(message_data, current_game_state)
                
                X_text_list.append(message_text)
                y_list.append(label)
                X_new_features_list.append(new_features)

    print(f"Loaded and processed {len(X_text_list)} samples from {filepath}.")
    return X_text_list, X_new_features_list, np.array(y_list)

# --- How to use it ---
# (Assuming 'game_state_lookup' is the dictionary you just built)

# 1. Load the data using the new function
print("Loading training data...")
X_train_text, X_train_features_list, y_train = load_and_flatten_data(os.path.join(DATA_PATH, "data", "raw", "2020_acl_diplomacy", "data", "train.jsonl"), game_state_lookup)

print("\nLoading validation data...")
X_val_text, X_val_features_list, y_val = load_and_flatten_data(os.path.join(DATA_PATH, "data", "raw", "2020_acl_diplomacy", "data", "validation.jsonl"), game_state_lookup)

print("\nLoading test data...")
X_test_text, X_test_features_list, y_test = load_and_flatten_data(os.path.join(DATA_PATH, "data", "raw", "2020_acl_diplomacy", "data", "test.jsonl"), game_state_lookup)


# 2. Check the output
print(f"\nExample of new features for first 5 training messages:")
for i in range(5):
    print(f"Message: '{X_train_text[i][:50]}...' | Features: {X_train_features_list[i]}")

Loading training data...
Loaded and processed 13132 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\train.jsonl.

Loading validation data...
Loaded and processed 1416 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\validation.jsonl.

Loading test data...
Loaded and processed 2741 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\test.jsonl.

Example of new features for first 5 training messages:
Message: 'Germany!

Just the person I want to speak with. I ...' | Features: {'sender_sc': 3, 'receiver_sc': 3, 'sender_unit_count': 0, 'receiver_unit_count': 0, 'score_delta': 0, 'are_adjacent': 0}
Message: 'You've whet my appetite, Italy. What's the suggest...' | Features: {'sender_sc': 3, 'receiver_sc': 3, 'sender_unit_count': 0, 'receiver_unit_count': 0, 'score_delta': 0, 'are_adjacent': 0}
Message: '👍...' | Features: {'sender_sc': 3, 'receiver_sc': 3, 'sen

In [32]:
import os
import re
import jsonlines

# --- 1. SET YOUR PATHS HERE ---
DATA_PATH = os.path.dirname(os.getcwd())
GAME_STATE_DIRECTORY = os.path.join(DATA_PATH, "data", "raw", "2020_acl_diplomacy", "moves")
TRAIN_FILE = os.path.join(DATA_PATH, "data", "raw", "2020_acl_diplomacy", "data", "train.jsonl")
# -----------------------------

print(f"--- Checking Game State Folder ---")
print(f"Looking in: {GAME_STATE_DIRECTORY}\n")

if not os.path.exists(GAME_STATE_DIRECTORY):
    print("Error: Directory not found. Please check the path.")
else:
    # 1. Check the first 10 files in the 'moves' folder
    print("First 10 files found in 'moves' folder:")
    file_pattern = re.compile(r"DiplomacyGame_(\d+)_(\d+)_(\w+)\.json")
    files_found = []
    
    for filename in os.listdir(GAME_STATE_DIRECTORY):
        if file_pattern.match(filename):
            files_found.append(filename)
        if len(files_found) >= 10:
            break
            
    if not files_found:
        print("  No files matching 'DiplomacyGame_...json' found.")
    else:
        for f in files_found:
            print(f"  {f}")

# 2. Check the first 10 game_ids in the 'train.jsonl' file
print(f"\n--- Checking 'train.jsonl' File ---")
print(f"Reading from: {TRAIN_FILE}\n")

try:
    with jsonlines.open(TRAIN_FILE, 'r') as reader:
        print("Game IDs needed by 'train.jsonl':")
        game_ids_needed = set()
        for i, game in enumerate(reader):
            game_ids_needed.add(game['game_id'])
            if i >= 10: # Just check the first few games
                break
        
        if not game_ids_needed:
            print("  No games found in 'train.jsonl'.")
        else:
            for game_id in game_ids_needed:
                print(f"  Needs GameID: {game_id}")
                
except FileNotFoundError:
    print(f"Error: '{TRAIN_FILE}' not found. Please check the path.")

--- Checking Game State Folder ---
Looking in: c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\moves

First 10 files found in 'moves' folder:
  No files matching 'DiplomacyGame_...json' found.

--- Checking 'train.jsonl' File ---
Reading from: c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\train.jsonl

Game IDs needed by 'train.jsonl':
  Needs GameID: 1


In [33]:
import jsonlines
import numpy as np
import re
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer # <-- Import the tool

# --- (Your create_game_state_dictionary function would be here) ---
# ...
# game_state_lookup = create_game_state_dictionary(...)

# --- (Your extract_features_from_state function would be here) ---
# ...

# --- (Your load_and_flatten_data function would be here) ---
# ...

# --- 1. Load All Data (as before) ---
print("Loading data...")
X_train_text, X_train_features_list, y_train = load_and_flatten_data(os.path.join(DATA_PATH, "data", "raw", "2020_acl_diplomacy", "data", "train.jsonl"), game_state_lookup)
X_val_text, X_val_features_list, y_val = load_and_flatten_data(os.path.join(DATA_PATH, "data", "raw", "2020_acl_diplomacy", "data", "validation.jsonl"), game_state_lookup)
X_test_text, X_test_features_list, y_test = load_and_flatten_data(os.path.join(DATA_PATH, "data", "raw", "2020_acl_diplomacy", "data", "test.jsonl"), game_state_lookup)

print("\nData loading complete.")

# --- 2. Convert Feature Dictionaries to a Numeric Matrix ---
print("Vectorizing new game state features...")

# Create the vectorizer
feature_vectorizer = DictVectorizer(sparse=False) # Use sparse=True for very large feature sets

# Fit ONLY on the training data
X_train_features_numeric = feature_vectorizer.fit_transform(X_train_features_list)

# Transform the validation and test data (DO NOT re-fit)
X_val_features_numeric = feature_vectorizer.transform(X_val_features_list)
X_test_features_numeric = feature_vectorizer.transform(X_test_features_list)

# --- 3. Check the Output ---
print("Feature vectorization complete.")
print(f"Names of new features: {feature_vectorizer.feature_names_}")
print(f"Shape of new training feature matrix: {X_train_features_numeric.shape}")

# Example:
if len(X_train_features_numeric) > 0:
    print(f"Example of first numeric feature row: {X_train_features_numeric[0]}")

Loading data...
Loaded and processed 13132 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\train.jsonl.
Loaded and processed 1416 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\validation.jsonl.
Loaded and processed 2741 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\test.jsonl.

Data loading complete.
Vectorizing new game state features...
Feature vectorization complete.
Names of new features: ['are_adjacent', 'receiver_sc', 'receiver_unit_count', 'score_delta', 'sender_sc', 'sender_unit_count']
Shape of new training feature matrix: (13132, 6)
Example of first numeric feature row: [0. 3. 0. 0. 3. 0.]


In [34]:
def evaluate_model(model_name, y_true, y_pred):
    print(f"\n=============================================")
    print(f"PERFORMANCE TEST: {model_name}")
    print(f"=============================================")
    
    f1_lie_class = f1_score(y_true, y_pred, pos_label=1, average='binary')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    
    print(f"F1 Score (Lie Class): {f1_lie_class:.4f}")
    print(f"F1 Score (Macro Avg): {f1_macro:.4f}")

    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print("           Predicted 0 (Truth)  |  Predicted 1 (Lie)")
    print(f"Actual 0:    {cm[0][0]:<20} | {cm[0][1]:<20}")
    print(f"Actual 1:    {cm[1][0]:<20} | {cm[1][1]:<20}")
    
    print("\nFull Classification Report:")
    print(classification_report(y_true, y_pred, target_names=['Class 0 (Truth)', 'Class 1 (Lie)']))

In [5]:
import json
import jsonlines
import numpy as np
# import spacy  <- REMOVED
# from spacy.lang.en import English <- REMOVED
import re # <- ADDED
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import os
import sys
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# --- 1. Global Settings & Initializer ---
# We will set TASK and POWER from command-line args, just like the original
# nlp = None # Will be initialized in main <- REMOVED

# --- 2. Helper Functions (from harbringers.py) ---

def regex_tokenizer(text): # <- RENAMED and RE-WRITTEN
    """
    Tokenizes text using regular expressions.
    This avoids the need for spacy.
    It finds all sequences of word characters (letters, numbers, underscore),
    which also effectively removes punctuation.
    """
    # \b matches a word boundary
    # \w+ matches one or more word characters (alphanumeric + underscore)
    return re.findall(r'\b\w+\b', text)

def aggregate(dataset):
    """Flattens dialogs into a list of single messages"""
    messages = []
    rec = []
    send = []
    power = []
    for dialogs in dataset:
        messages.extend(dialogs['messages'])
        rec.extend(dialogs['receiver_labels'])
        send.extend(dialogs['sender_labels'])
        power.extend(dialogs['game_score_delta'])
    
    merged = []
    for i, item in enumerate(messages):
        merged.append({
            'message': item,
            'sender_annotation': send[i],
            'receiver_annotation': rec[i],
            'score_delta': int(power[i])
        })
    return merged

def convert_to_features(dataset, feature_dict, TASK, POWER):
    """
    This is the core of harbringers.py. It converts a message
    into a vector of lexicon-based features.
    """
    processed_data = []
    
    for message in dataset:
        
        annotation_key = 'sender_annotation' if TASK == "SENDER" else 'receiver_annotation'
        
        # Skip messages with no label
        if message[annotation_key] == "NOANNOTATION":
            continue

        features = []
        
        # 1. Lexicon Features
        tokenized_message = regex_tokenizer(message['message'].lower()) # <- USE NEW FUNCTION
        
        for feature_category in feature_dict.keys():
            total_count = 0
            for word in feature_dict[feature_category]:
                # Count all occurrences of this word
                total_count += tokenized_message.count(word)
            features.append(total_count)
        
        # 2. Power Features
        if POWER == "y":
            features.append(1 if message['score_delta'] > 4 else 0)
            features.append(1 if message['score_delta'] < -4 else 0)
        
        # 3. Label (Appended last)
        # We'll use our convention: Lie = 1, True = 0
        if message[annotation_key] == False:
            features.append(1) # Lie
        else:
            features.append(0) # True
            
        processed_data.append(features)
        
    return processed_data

def split_xy(data):
    """Splits a list of lists into X (features) and y (labels)"""
    X, y = [], []
    for line in data:
        X.append(line[:-1])  # All but the last item
        y.append(line[-1])   # The last item
    return np.array(X), np.array(y)

def evaluate_model(model_name, y_true, y_pred):
    """Prints a detailed performance report"""
    print(f"\n=============================================")
    print(f"PERFORMANCE TEST: {model_name}")
    print(f"=============================================")
    
    # Use pos_label=1 to get F1 for the "Lie" class
    f1_lie_class = f1_score(y_true, y_pred, pos_label=1, average='binary')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    
    print(f"F1 Score (Lie Class): {f1_lie_class:.4f}")
    print(f"F1 Score (Macro Avg): {f1_macro:.4f}")

    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print("           Predicted 0 (Truth)  |  Predicted 1 (Lie)")
    print(f"Actual 0:    {cm[0][0]:<20} | {cm[0][1]:<20}")
    print(f"Actual 1:    {cm[1][0]:<20} | {cm[1][1]:<20}")
    
    print("\nFull Classification Report:")
    # Set zero_division=0 to avoid errors if a class is never predicted
    print(classification_report(y_true, y_pred, target_names=['Class 0 (Truth)', 'Class 1 (Lie)'], zero_division=0))

# --- 3. MAIN EXECUTION ---
if __name__ == '__main__':
    
    # --- Parse Command-Line Arguments (like harbringers.py) ---
    # This script defaults to 'sender' (s) and 'power' (y)
    # You can run: python test_lexicon_model.py s n  (to turn off power)
    # Or:          python test_lexicon_model.py r y  (to predict receiver labels)
    
    TASK = "SENDER"
    POWER = "y"
    
    if len(sys.argv) >= 2:
        if sys.argv[1] == 's':
            TASK = "SENDER"
        elif sys.argv[1] == 'r':
            TASK = "RECEIVER"
    if len(sys.argv) >= 3:
        if sys.argv[2] == 'n':
            POWER = 'n'
    
    print(f"Running model for TASK={TASK}, POWER={POWER}")
    
    # --- !! IMPORTANT !! ---
    # Make sure these paths are correct for your project structure
    DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "data")
    LEXICON_PATH = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "utils", "2015_Diplomacy_lexicon.json")
    # -----------------------

    # 1. Load Lexicon
    print(f"Loading lexicon from {LEXICON_PATH}...")
    try:
        with open(LEXICON_PATH, 'r') as f:
            feature_dict = json.loads(f.readline())
        
        # Add custom features from the original script
        feature_dict['but'] = ['but']
        feature_dict['countries'] = ['austria', 'england', 'france', 'germany', 'italy', 'russia', 'turkey']
        print(f"Successfully loaded lexicon with {len(feature_dict)} feature categories.")
    except Exception as e:
        print(f"CRITICAL ERROR: Could not load lexicon file at {LEXICON_PATH}")
        print(f"Please ensure the file exists.")
        print(f"Error: {e}")
        sys.exit(1) # Exit the script if lexicon fails

    # 2. Load Datasets
    print("Loading datasets...")
    try:
        with jsonlines.open(os.path.join(DATA_PATH, 'train.jsonl'), 'r') as reader:
            train_data_raw = list(reader)
        with jsonlines.open(os.path.join(DATA_PATH, 'test.jsonl'), 'r') as reader:
            test_data_raw = list(reader)
    except FileNotFoundError:
        print(f"CRITICAL ERROR: Could not find data files in '{DATA_PATH}'")
        print("Please ensure DATA_PATH is set correctly.")
        sys.exit(1)

    # 3. Process Data (Aggregate -> Convert to Features -> Split)
    print("Aggregating messages...")
    train_agg = aggregate(train_data_raw)
    test_agg = aggregate(test_data_raw)
    
    print("Converting data to lexicon features...")
    train_converted = convert_to_features(train_agg, feature_dict, TASK, POWER)
    test_converted = convert_to_features(test_agg, feature_dict, TASK, POWER)
    
    print("Splitting features and labels...")
    X_train, y_train = split_xy(train_converted)
    X_test, y_test = split_xy(test_converted)
    
    if len(X_train) == 0 or len(X_test) == 0:
        print("CRITICAL ERROR: No data was processed.")
        print("This might happen if 'NOANNOTATION' is the only label.")
        sys.exit(1)

    # 4. Scale Features
    print("Scaling features...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"Feature matrix shape: {X_train_scaled.shape}")
    print(f"Training samples: {len(y_train)}, Test samples: {len(y_test)}")

    # 5. Train Model
    print("Training Logistic Regression model...")
    # class_weight='balanced' is CRITICAL for this imbalanced dataset
    logmodel = LogisticRegression(class_weight='balanced', max_iter=1000)
    logmodel.fit(X_train_scaled, y_train)

    # 6. Evaluate
    print("Evaluating model on test set...")
    y_pred = logmodel.predict(X_test_scaled)
    evaluate_model(f"Logistic Regression (Lexicon, {TASK})", y_test, y_pred)

Running model for TASK=SENDER, POWER=y
Loading lexicon from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\utils\2015_Diplomacy_lexicon.json...
Successfully loaded lexicon with 10 feature categories.
Loading datasets...
Aggregating messages...
Converting data to lexicon features...
Splitting features and labels...
Scaling features...
Feature matrix shape: (13132, 12)
Training samples: 13132, Test samples: 2741
Training Logistic Regression model...
Evaluating model on test set...

PERFORMANCE TEST: Logistic Regression (Lexicon, SENDER)
F1 Score (Lie Class): 0.2347
F1 Score (Macro Avg): 0.5323

Confusion Matrix:
           Predicted 0 (Truth)  |  Predicted 1 (Lie)
Actual 0:    1861                 | 640                 
Actual 1:    123                  | 117                 

Full Classification Report:
                 precision    recall  f1-score   support

Class 0 (Truth)       0.94      0.74      0.83      2501
  Class 1 (Lie)       0.15      0.49      

In [10]:
# Ensure TensorFlow is installed in the notebook environment
%pip install tensorflow --quiet

import json
import jsonlines
import numpy as np
import re
from collections import defaultdict
import os
import sys
import warnings
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# --- 1. NEW: EXPERT KNOWLEDGE MAPPING ---

# This map is our "secret weapon". It connects text ("paris") to game data ("Par").
NAME_TO_ABBREVIATION = {
    'north atlantic ocean': 'Nao', 'nao': 'Nao',
    'irish sea': 'Iri', 'iri': 'Iri',
    'english channel': 'Eng', 'eng': 'Eng', 'channel': 'Eng',
    'mid-atlantic ocean': 'Mao', 'mao': 'Mao', 'mid atlantic': 'Mao',
    'north sea': 'Nth', 'nth': 'Nth',
    'norwegian sea': 'Nwg', 'nwg': 'Nwg',
    'barents sea': 'Bar', 'bar': 'Bar',
    'helgoland bight': 'Hel', 'hel': 'Hel',
    'skagerrak': 'Ska', 'ska': 'Ska',
    'baltic sea': 'Bal', 'bal': 'Bal',
    'gulf of bothnia': 'Bot', 'bot': 'Bot',
    'western mediterranean': 'Wes', 'wes': 'Wes',
    'gulf of lyon': 'Lyo', 'lyo': 'Lyo',
    'tyrrhenian sea': 'Tyn', 'tyn': 'Tyn',
    'ionian sea': 'Ion', 'ion': 'Ion',
    'adriatic sea': 'Adr', 'adr': 'Adr',
    'aegean sea': 'Aeg', 'aeg': 'Aeg',
    'eastern mediterranean': 'Eas', 'eas': 'Eas',
    'black sea': 'Bla', 'bla': 'Bla',
    
    'clyde': 'Cly', 'cly': 'Cly',
    'edinburgh': 'Edi', 'edi': 'Edi',
    'liverpool': 'Lvp', 'lvp': 'Lvp',
    'yorkshire': 'Yor', 'yor': 'Yor',
    'wales': 'Wal', 'wal': 'Wal',
    'london': 'Lon', 'lon': 'Lon',
    'belgium': 'Bel', 'bel': 'Bel',
    'holland': 'Hol', 'hol': 'Hol',
    'denmark': 'Den', 'den': 'Den',
    'sweden': 'Swe', 'swe': 'Swe',
    'norway': 'Nwy', 'nwy': 'Nwy',
    'finland': 'Fin', 'fin': 'Fin',
    'st petersburg': 'Stp', 'stp': 'Stp',
    'livonia': 'Lvn', 'lvn': 'Lvn',
    'prussia': 'Pru', 'pru': 'Pru',
    'kiel': 'Kie', 'kie': 'Kie',
    'berlin': 'Ber', 'ber': 'Ber',
    'silesia': 'Sil', 'sil': 'Sil',
    'poland': 'War', 'war': 'War', 'warsaw': 'War',
    'moscow': 'Mos', 'mos': 'Mos',
    'ukraine': 'Ukr', 'ukr': 'Ukr',
    'sevastopol': 'Sev', 'sev': 'Sev',
    'ruhr': 'Ruh', 'ruh': 'Ruh',
    'munich': 'Mun', 'mun': 'Mun',
    'bohemia': 'Boh', 'boh': 'Boh',
    'galicia': 'Gal', 'gal': 'Gal',
    'brest': 'Bre', 'bre': 'Bre',
    'picardy': 'Pic', 'pic': 'Pic',
    'paris': 'Par', 'par': 'Par',
    'burgundy': 'Bur', 'bur': 'Bur',
    'gascony': 'Gas', 'gas': 'Gas',
    'marseilles': 'Mar', 'mar': 'Mar',
    'spain': 'Spa', 'spa': 'Spa',
    'portugal': 'Por', 'por': 'Por',
    'piedmont': 'Pie', 'pie': 'Pie',
    'tyrolia': 'Tyr', 'tyr': 'Tyr',
    'vienna': 'Vie', 'vie': 'Vie',
    'trieste': 'Tri', 'tri': 'Tri',
    'budapest': 'Bud', 'bud': 'Bud',
    'rumania': 'Rum', 'rum': 'Rum',
    'serbia': 'Ser', 'ser': 'Ser',
    'venice': 'Ven', 'ven': 'Ven',
    'rome': 'Rom', 'rom': 'Rom',
    'apulia': 'Apu', 'apu': 'Apu',
    'naples': 'Nap', 'nap': 'Nap',
    'albania': 'Alb', 'alb': 'Alb',
    'greece': 'Gre', 'gre': 'Gre',
    'bulgaria': 'Bul', 'bul': 'Bul',
    'constantinople': 'Con', 'con': 'Con',
    'ankara': 'Ank', 'ank': 'Ank',
    'armenia': 'Arm', 'arm': 'Arm',
    'smyrna': 'Smy', 'smy': 'Smy',
    'syria': 'Syr', 'syr': 'Syr',
    'north africa': 'Naf', 'naf': 'Naf',
    'tunis': 'Tun', 'tun': 'Tun'
}

# This is a helper set for the new tokenizer
# We find all unique "words" in our map's keys
# e.g., "north", "atlantic", "ocean", "nao", "irish", "sea", "iri", etc.
TEXT_LOOKUP_WORDS = set()
for key in NAME_TO_ABBREVIATION.keys():
    TEXT_LOOKUP_WORDS.update(key.split())


# --- 2. GAME STATE LOADING (Same as before) ---
def create_game_state_dictionary(game_state_dir):
    filename_pattern = re.compile(r"DiplomacyGame(\d+)_(\d+)_(\w+)\.json")
    game_states = defaultdict(lambda: defaultdict(dict))
    print(f"Starting to build game state dictionary from: {game_state_dir}")
    file_count = 0
    if not os.path.exists(game_state_dir):
        print(f"Error: Directory not found: {game_state_dir}")
        return None
    for filename in os.listdir(game_state_dir):
        match = filename_pattern.match(filename)
        if match:
            game_id = int(match.group(1))
            year = match.group(2)
            season = match.group(3).lower() # <-- Store key as lowercase
            file_path = os.path.join(game_state_dir, filename)
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)
                    game_states[game_id][year][season] = data
                    file_count += 1
            except Exception as e:
                print(f"Warning: Could not load or parse {filename}. Error: {e}")
    print(f"Successfully loaded and indexed {file_count} game state files.")
    return game_states

# --- 3. NEW: TEXT-TO-STATE FEATURE EXTRACTION ---

def find_mentioned_territories_abbreviations(message_text, lookup_map, lookup_words):
    """
    Finds all unique territory ABBREVIATIONS (e.g., 'Par') mentioned
    in a raw text message (e.g., "I'm going to Paris").
    """
    # Tokenize the message by finding all words
    words = re.findall(r'\b\w+\b', message_text.lower())
    
    # Filter for words that are part of our map to speed things up
    potential_words = [w for w in words if w in lookup_words]
    
    if not potential_words:
        return set()

    text = " ".join(potential_words)
    mentioned_abbreviations = set()
    
    # Check for multi-word keys first (e.g., "north atlantic ocean")
    for name, abbr in lookup_map.items():
        if " " in name and name in text:
            mentioned_abbreviations.add(abbr)
            
    # Check for single-word keys (e.g., "paris" or "par")
    for word in potential_words:
        if word in lookup_map:
            mentioned_abbreviations.add(lookup_map[word])
            
    return mentioned_abbreviations

def extract_features_from_state(message_data, game_state):
    """
    The master feature function. Creates all game-state and
    text-to-state contradiction features.
    """
    new_features = {}
    
    sender = message_data['speaker'].lower()
    receiver = message_data['receiver'].lower()
    message_text = message_data['text']
    
    # --- Get state data ---
    territories_data = game_state.get('territories', {})
    orders_data = game_state.get('orders', {})
    sc_text = game_state.get('sc', '')
    
    # === 1. & 2. SC and Unit Counts (Same as before) ===
    sc_pattern = re.compile(r"\s*(\w+)\s+(\d+)\s*")
    all_scores = {}
    for match in sc_pattern.finditer(sc_text):
        country = match.group(1).lower()
        score = int(match.group(2))
        all_scores[country] = score
    new_features['sender_sc'] = all_scores.get(sender, 0)
    new_features['receiver_sc'] = all_scores.get(receiver, 0)

    sender_units_orders = orders_data.get(sender.upper(), {})
    receiver_units_orders = orders_data.get(receiver.upper(), {})
    new_features['sender_unit_count'] = len(sender_units_orders)
    new_features['receiver_unit_count'] = len(receiver_units_orders)
    
    # === 3. Score Delta (Same as before) ===
    new_features['score_delta'] = message_data['score_delta']
    
    # === 4. Adjacency (Same as before) ===
    # This requires a global DIPLOMACY_MAP, which we'll define.
    # We'll just hard-code this feature for now to avoid the map.
    # is_adjacent = are_players_adjacent(sender, receiver, territories_data, DIPLOMACY_MAP)
    # new_features['are_adjacent'] = 1 if is_adjacent else 0
    
    # === 5. NEW: Text-to-State & Contradiction Features ===
    
    # Find all territories mentioned in the message
    mentioned_abbreviations = find_mentioned_territories_abbreviations(
        message_text, NAME_TO_ABBREVIATION, TEXT_LOOKUP_WORDS
    )
    new_features['territory_mention_count'] = len(mentioned_abbreviations)

    # Find all territories owned by sender and receiver
    sender_owned_territories = set()
    receiver_owned_territories = set()
    for territory, owner in territories_data.items():
        if owner.lower() == sender:
            sender_owned_territories.add(territory.upper()) # Abbreviations are case-insensitive
        elif owner.lower() == receiver:
            receiver_owned_territories.add(territory.upper())
            
    # Find all territories involved in sender's orders
    origin_territories = set(sender_units_orders.keys()) # e.g., {'Arm', 'Bul', 'Ank'}
    target_territories = set()
    for unit, order in sender_units_orders.items():
        if 'to' in order:
            target_territories.add(order['to'].upper()) # e.g., {'Sev', 'Rum'}

    # Create the new features (as 1s or 0s)
    new_features['mentions_own_territory'] = 0
    new_features['mentions_receiver_territory'] = 0
    new_features['mentions_origin_unit'] = 0
    new_features['mentions_target_territory'] = 0
    
    for abbr in mentioned_abbreviations:
        abbr_upper = abbr.upper()
        if abbr_upper in sender_owned_territories:
            new_features['mentions_own_territory'] = 1
        if abbr_upper in receiver_owned_territories:
            new_features['mentions_receiver_territory'] = 1
        if abbr_upper in origin_territories:
            new_features['mentions_origin_unit'] = 1
        if abbr_upper in target_territories:
            new_features['mentions_target_territory'] = 1
            
    return new_features

# --- 4. DATA LOADING ---

def load_and_flatten_data(filepath, game_state_lookup):
    X_text_list = []
    y_list = []
    X_new_features_list = [] 

    with jsonlines.open(filepath, 'r') as reader:
        for game in reader:
            game_id = game['game_id']
            years = game['years']
            seasons = game['seasons']
            speakers = game['speakers']
            
            for i in range(len(game['messages'])):
                sender_label = game['sender_labels'][i]
                if sender_label == "NOANNOTATION":
                    continue
                
                label = 1 if sender_label == False else 0
                message_text = game['messages'][i]
                year = years[i]
                season = seasons[i].lower() # <-- Use lowercase for lookup
                
                try:
                    current_game_state = game_state_lookup[game_id][year][season]
                except KeyError:
                    continue
                
                message_data = {
                    'text': message_text,
                    'speaker': speakers[i],
                    'receiver': game['receivers'][i],
                    'score_delta': int(game['game_score_delta'][i])
                }
                
                new_features = extract_features_from_state(message_data, current_game_state)
                
                X_text_list.append(message_text)
                y_list.append(label)
                X_new_features_list.append(new_features)

    print(f"Loaded and processed {len(X_text_list)} samples from {filepath}.")
    return X_text_list, X_new_features_list, np.array(y_list)

# --- 5. MODEL EVALUATION ---

def evaluate_model(model_name, y_true, y_pred_probs): # y_pred_probs is an array of floats
    """Prints a detailed performance report"""
    
    print(f"\n=============================================")
    print(f"PERFORMANCE TEST: {model_name}")
    print(f"=============================================")
    
    # --- THIS IS THE FIX ---
    # Convert continuous probabilities (e.g., 0.73) into binary
    # class labels (e.g., 1) using a 0.5 threshold.
    y_pred = (y_pred_probs > 0.5).astype(int)
    
    # Now, y_true and y_pred are both binary (0s and 1s)
    f1_lie_class = f1_score(y_true, y_pred, pos_label=1, average='binary')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    
    print(f"F1 Score (Lie Class): {f1_lie_class:.4f}")
    print(f"F1 Score (Macro Avg): {f1_macro:.4f}")

    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred) # Use the binary y_pred
    print("           Predicted 0 (Truth)  |  Predicted 1 (Lie)")
    print(f"Actual 0:    {cm[0][0]:<20} | {cm[0][1]:<20}")
    print(f"Actual 1:    {cm[1][0]:<20} | {cm[1][1]:<20}")
    
    print("\nFull Classification Report:")
    print(classification_report(y_true, y_pred, target_names=['Class 0 (Truth)', 'Class 1 (Lie)'], zero_division=0))

# --- 6. NEW: NEURAL NETWORK PIPELINE ---
if __name__ == '__main__':
    
    # --- Set Paths ---
    GAME_STATE_DIRECTORY = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "moves") 

    TRAIN_FILE = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "data", "train.jsonl") 
    VAL_FILE = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "data", "validation.jsonl") 
    TEST_FILE = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "data", "test.jsonl") 
    
    VOCAB_SIZE = 10000
    MAX_LEN = 120
    EMBEDDING_DIM = 64
    LSTM_UNITS = 64

    # --- Load Game State Dictionary ---
    game_state_lookup = create_game_state_dictionary(GAME_STATE_DIRECTORY)
    if not game_state_lookup:
        print("Game state lookup dictionary is empty. Exiting.")
        sys.exit(1)
        
    # --- Load all data ---
    print("Loading all datasets...")
    X_train_text, X_train_features_list, y_train = load_and_flatten_data(TRAIN_FILE, game_state_lookup)
    X_val_text, X_val_features_list, y_val = load_and_flatten_data(VAL_FILE, game_state_lookup)
    X_test_text, X_test_features_list, y_test = load_and_flatten_data(TEST_FILE, game_state_lookup)

    if len(X_train_text) == 0:
        print("CRITICAL ERROR: No training data was loaded.")
        sys.exit(1)

    # --- 1. PREPARE TEXT INPUT (Tokenizer + Padding) ---
    print("\nPreprocessing Text Data...")
    tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train_text) # Fit ONLY on training text
    
    X_train_seq = tokenizer.texts_to_sequences(X_train_text)
    X_val_seq = tokenizer.texts_to_sequences(X_val_text)
    X_test_seq = tokenizer.texts_to_sequences(X_test_text)
    
    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')
    
    print(f"Text data shape: {X_train_pad.shape}")

    # --- 2. PREPARE GAME STATE INPUT (DictVectorizer + Scaling) ---
    print("Preprocessing Game State Data...")
    feature_vectorizer = DictVectorizer(sparse=False)  # Use dense arrays instead of sparse
    X_train_features_numeric = feature_vectorizer.fit_transform(X_train_features_list)
    X_val_features_numeric = feature_vectorizer.transform(X_val_features_list)
    X_test_features_numeric = feature_vectorizer.transform(X_test_features_list)
    
    scaler = StandardScaler()  # Now works with dense arrays
    X_train_features_scaled = scaler.fit_transform(X_train_features_numeric)
    X_val_features_scaled = scaler.transform(X_val_features_numeric)
    X_test_features_scaled = scaler.transform(X_test_features_numeric)
    
    num_game_features = X_train_features_scaled.shape[1]
    print(f"Game state feature names: {feature_vectorizer.feature_names_}")
    print(f"Game state data shape: {X_train_features_scaled.shape}")

    # --- 3. DEFINE THE DUAL-INPUT MODEL ---
    print("\nBuilding Neural Network Model...")
    
    # Input 1: Text Sequences
    text_input_layer = Input(shape=(MAX_LEN,), name='text_input')
    text_model = Embedding(input_dim=VOCAB_SIZE+1, output_dim=EMBEDDING_DIM, input_length=MAX_LEN)(text_input_layer)
    text_model = SpatialDropout1D(0.2)(text_model) # Dropout for embeddings
    text_model = LSTM(LSTM_UNITS, dropout=0.2, recurrent_dropout=0.2, name='lstm_layer')(text_model)
    
    # Input 2: Game State Features
    game_state_input_layer = Input(shape=(num_game_features,), name='game_state_input')
    
    # Combine (Concatenate)
    combined = concatenate([text_model, game_state_input_layer], name='concatenate')
    
    # Classifier Head
    output = Dense(64, activation='relu', name='dense_1')(combined)
    output = Dropout(0.5, name='dropout')(output)
    output = Dense(1, activation='sigmoid', name='output_layer')(output)
    
    model = Model(inputs=[text_input_layer, game_state_input_layer], outputs=output)
    
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    model.summary()

    # --- 4. HANDLE CLASS IMBALANCE ---
    # This is CRITICAL for getting a good F1 score
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    class_weight_dict = dict(enumerate(class_weights))
    print(f"Class Weights (for balancing): {class_weight_dict}")

    # --- 5. TRAIN THE MODEL ---
    print("\nTraining the model...")
    
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=3,         # Stop after 3 epochs with no improvement
        restore_best_weights=True # Keep the best model, not the last
    )
    
    history = model.fit(
        [X_train_pad, X_train_features_scaled], # Pass inputs as a list
        y_train,
        epochs=15,
        batch_size=64,
        validation_data=([X_val_pad, X_val_features_scaled], y_val),
        callbacks=[early_stopping],
        class_weight=class_weight_dict # Apply the weights
    )

    # --- 6. EVALUATE THE FINAL MODEL ---
    print("\nEvaluating final model on test set...")
    y_pred_probs = model.predict([X_test_pad, X_test_features_scaled])
    evaluate_model("Dual-Input Neural Network", y_test, y_pred_probs)

Note: you may need to restart the kernel to use updated packages.
Starting to build game state dictionary from: c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\moves
Successfully loaded and indexed 342 game state files.
Loading all datasets...
Loaded and processed 13132 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\train.jsonl.
Loaded and processed 1416 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\validation.jsonl.
Loaded and processed 2741 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\test.jsonl.

Preprocessing Text Data...
Loaded and processed 13132 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\train.jsonl.
Loaded and processed 1416 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\validation.jsonl.
Loaded and processed 274

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 120)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 120, 64)   │    640,064 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_1 │ (None, 120, 64)   │          0 │ embedding_1[0][0] │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_layer (LSTM)   │ (None, 64)        │     33,024 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ game_state_input    │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 74)        │          0 │ lstm_layer[0][0], │
│ (Concatenate)       │                   │            │ game_state_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      4,800 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 1)         │         65 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 677,953 (2.59 MB)

 Trainable params: 677,953 (2.59 MB)

 Non-trainable params: 0 (0.00 B)

Class Weights (for balancing): {0: np.float64(0.5235627142971055), 1: np.float64(11.109983079526227)}

Training the model...
Epoch 1/15
Epoch 1/15
206/206 ━━━━━━━━━━━━━━━━━━━━ 19s 77ms/step - accuracy: 0.6430 - loss: 0.6821 - val_accuracy: 0.5395 - val_loss: 0.7376
Epoch 2/15
206/206 ━━━━━━━━━━━━━━━━━━━━ 19s 77ms/step - accuracy: 0.6430 - loss: 0.6821 - val_accuracy: 0.5395 - val_loss: 0.7376
Epoch 2/15
206/206 ━━━━━━━━━━━━━━━━━━━━ 18s 86ms/step - accuracy: 0.6550 - loss: 0.6664 - val_accuracy: 0.7874 - val_loss: 0.6766
Epoch 3/15
206/206 ━━━━━━━━━━━━━━━━━━━━ 18s 86ms/step - accuracy: 0.6550 - loss: 0.6664 - val_accuracy: 0.7874 - val_loss: 0.6766
Epoch 3/15
206/206 ━━━━━━━━━━━━━━━━━━━━ 18s 87ms/step - accuracy: 0.6643 - loss: 0.6721 - val_accuracy: 0.6384 - val_loss: 0.6981
Epoch 4/15
206/206 ━━━━━━━━━━━━━━━━━━━━ 18s 87ms/step - accuracy: 0.6643 - loss: 0.6721 - val_accuracy: 0.6384 - val_loss: 0.6981
Epoch 4/15
206/206 ━━━━━━━━━━━━━━━━━━━━ 19s 91ms/step - accuracy: 0.6525 - loss: 0.6

In [ ]:
import json
import jsonlines
import numpy as np
import re
from collections import defaultdict
import os
import sys
import warnings
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# --- 1. NEW: EXPERT KNOWLEDGE MAPPING ---

# This map is our "secret weapon". It connects text ("paris") to game data ("Par").
NAME_TO_ABBREVIATION = {
    'north atlantic ocean': 'Nao', 'nao': 'Nao',
    'irish sea': 'Iri', 'iri': 'Iri',
    'english channel': 'Eng', 'eng': 'Eng', 'channel': 'Eng',
    'mid-atlantic ocean': 'Mao', 'mao': 'Mao', 'mid atlantic': 'Mao',
    'north sea': 'Nth', 'nth': 'Nth',
    'norwegian sea': 'Nwg', 'nwg': 'Nwg',
    'barents sea': 'Bar', 'bar': 'Bar',
    'helgoland bight': 'Hel', 'hel': 'Hel',
    'skagerrak': 'Ska', 'ska': 'Ska',
    'baltic sea': 'Bal', 'bal': 'Bal',
    'gulf of bothnia': 'Bot', 'bot': 'Bot',
    'western mediterranean': 'Wes', 'wes': 'Wes',
    'gulf of lyon': 'Lyo', 'lyo': 'Lyo',
    'tyrrhenian sea': 'Tyn', 'tyn': 'Tyn',
    'ionian sea': 'Ion', 'ion': 'Ion',
    'adriatic sea': 'Adr', 'adr': 'Adr',
    'aegean sea': 'Aeg', 'aeg': 'Aeg',
    'eastern mediterranean': 'Eas', 'eas': 'Eas',
    'black sea': 'Bla', 'bla': 'Bla',
    
    'clyde': 'Cly', 'cly': 'Cly',
    'edinburgh': 'Edi', 'edi': 'Edi',
    'liverpool': 'Lvp', 'lvp': 'Lvp',
    'yorkshire': 'Yor', 'yor': 'Yor',
    'wales': 'Wal', 'wal': 'Wal',
    'london': 'Lon', 'lon': 'Lon',
    'belgium': 'Bel', 'bel': 'Bel',
    'holland': 'Hol', 'hol': 'Hol',
    'denmark': 'Den', 'den': 'Den',
    'sweden': 'Swe', 'swe': 'Swe',
    'norway': 'Nwy', 'nwy': 'Nwy',
    'finland': 'Fin', 'fin': 'Fin',
    'st petersburg': 'Stp', 'stp': 'Stp',
    'livonia': 'Lvn', 'lvn': 'Lvn',
    'prussia': 'Pru', 'pru': 'Pru',
    'kiel': 'Kie', 'kie': 'Kie',
    'berlin': 'Ber', 'ber': 'Ber',
    'silesia': 'Sil', 'sil': 'Sil',
    'poland': 'War', 'war': 'War', 'warsaw': 'War',
    'moscow': 'Mos', 'mos': 'Mos',
    'ukraine': 'Ukr', 'ukr': 'Ukr',
    'sevastopol': 'Sev', 'sev': 'Sev',
    'ruhr': 'Ruh', 'ruh': 'Ruh',
    'munich': 'Mun', 'mun': 'Mun',
    'bohemia': 'Boh', 'boh': 'Boh',
    'galicia': 'Gal', 'gal': 'Gal',
    'brest': 'Bre', 'bre': 'Bre',
    'picardy': 'Pic', 'pic': 'Pic',
    'paris': 'Par', 'par': 'Par',
    'burgundy': 'Bur', 'bur': 'Bur',
    'gascony': 'Gas', 'gas': 'Gas',
    'marseilles': 'Mar', 'mar': 'Mar',
    'spain': 'Spa', 'spa': 'Spa',
    'portugal': 'Por', 'por': 'Por',
    'piedmont': 'Pie', 'pie': 'Pie',
    'tyrolia': 'Tyr', 'tyr': 'Tyr',
    'vienna': 'Vie', 'vie': 'Vie',
    'trieste': 'Tri', 'tri': 'Tri',
    'budapest': 'Bud', 'bud': 'Bud',
    'rumania': 'Rum', 'rum': 'Rum',
    'serbia': 'Ser', 'ser': 'Ser',
    'venice': 'Ven', 'ven': 'Ven',
    'rome': 'Rom', 'rom': 'Rom',
    'apulia': 'Apu', 'apu': 'Apu',
    'naples': 'Nap', 'nap': 'Nap',
    'albania': 'Alb', 'alb': 'Alb',
    'greece': 'Gre', 'gre': 'Gre',
    'bulgaria': 'Bul', 'bul': 'Bul',
    'constantinople': 'Con', 'con': 'Con',
    'ankara': 'Ank', 'ank': 'Ank',
    'armenia': 'Arm', 'arm': 'Arm',
    'smyrna': 'Smy', 'smy': 'Smy',
    'syria': 'Syr', 'syr': 'Syr',
    'north africa': 'Naf', 'naf': 'Naf',
    'tunis': 'Tun', 'tun': 'Tun'
}

# This is a helper set for the new tokenizer
# We find all unique "words" in our map's keys
# e.g., "north", "atlantic", "ocean", "nao", "irish", "sea", "iri", etc.
TEXT_LOOKUP_WORDS = set()
for key in NAME_TO_ABBREVIATION.keys():
    TEXT_LOOKUP_WORDS.update(key.split())


# --- 2. GAME STATE LOADING (Same as before) ---
def create_game_state_dictionary(game_state_dir):
    filename_pattern = re.compile(r"DiplomacyGame(\d+)_(\d+)_(\w+)\.json")
    game_states = defaultdict(lambda: defaultdict(dict))
    print(f"Starting to build game state dictionary from: {game_state_dir}")
    file_count = 0
    if not os.path.exists(game_state_dir):
        print(f"Error: Directory not found: {game_state_dir}")
        return None
    for filename in os.listdir(game_state_dir):
        match = filename_pattern.match(filename)
        if match:
            game_id = int(match.group(1))
            year = match.group(2)
            season = match.group(3).lower() # <-- Store key as lowercase
            file_path = os.path.join(game_state_dir, filename)
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)
                    game_states[game_id][year][season] = data
                    file_count += 1
            except Exception as e:
                print(f"Warning: Could not load or parse {filename}. Error: {e}")
    print(f"Successfully loaded and indexed {file_count} game state files.")
    return game_states

# --- 3. NEW: TEXT-TO-STATE FEATURE EXTRACTION ---

def find_mentioned_territories_abbreviations(message_text, lookup_map, lookup_words):
    """
    Finds all unique territory ABBREVIATIONS (e.g., 'Par') mentioned
    in a raw text message (e.g., "I'm going to Paris").
    """
    # Tokenize the message by finding all words
    words = re.findall(r'\b\w+\b', message_text.lower())
    
    # Filter for words that are part of our map to speed things up
    potential_words = [w for w in words if w in lookup_words]
    
    if not potential_words:
        return set()

    text = " ".join(potential_words)
    mentioned_abbreviations = set()
    
    # Check for multi-word keys first (e.g., "north atlantic ocean")
    for name, abbr in lookup_map.items():
        if " " in name and name in text:
            mentioned_abbreviations.add(abbr)
            
    # Check for single-word keys (e.g., "paris" or "par")
    for word in potential_words:
        if word in lookup_map:
            mentioned_abbreviations.add(lookup_map[word])
            
    return mentioned_abbreviations

def extract_features_from_state(message_data, game_state):
    """
    The master feature function. Creates all game-state and
    text-to-state contradiction features.
    """
    new_features = {}
    
    sender = message_data['speaker'].lower()
    receiver = message_data['receiver'].lower()
    message_text = message_data['text']
    
    # --- Get state data ---
    territories_data = game_state.get('territories', {})
    orders_data = game_state.get('orders', {})
    sc_text = game_state.get('sc', '')
    
    # === 1. & 2. SC and Unit Counts (Same as before) ===
    sc_pattern = re.compile(r"\s*(\w+)\s+(\d+)\s*")
    all_scores = {}
    for match in sc_pattern.finditer(sc_text):
        country = match.group(1).lower()
        score = int(match.group(2))
        all_scores[country] = score
    new_features['sender_sc'] = all_scores.get(sender, 0)
    new_features['receiver_sc'] = all_scores.get(receiver, 0)

    sender_units_orders = orders_data.get(sender.upper(), {})
    receiver_units_orders = orders_data.get(receiver.upper(), {})
    new_features['sender_unit_count'] = len(sender_units_orders)
    new_features['receiver_unit_count'] = len(receiver_units_orders)
    
    # === 3. Score Delta (Same as before) ===
    new_features['score_delta'] = message_data['score_delta']
    
    # === 4. Adjacency (Same as before) ===
    # This requires a global DIPLOMACY_MAP, which we'll define.
    # We'll just hard-code this feature for now to avoid the map.
    # is_adjacent = are_players_adjacent(sender, receiver, territories_data, DIPLOMACY_MAP)
    # new_features['are_adjacent'] = 1 if is_adjacent else 0
    
    # === 5. NEW: Text-to-State & Contradiction Features ===
    
    # Find all territories mentioned in the message
    mentioned_abbreviations = find_mentioned_territories_abbreviations(
        message_text, NAME_TO_ABBREVIATION, TEXT_LOOKUP_WORDS
    )
    new_features['territory_mention_count'] = len(mentioned_abbreviations)

    # Find all territories owned by sender and receiver
    sender_owned_territories = set()
    receiver_owned_territories = set()
    for territory, owner in territories_data.items():
        if owner.lower() == sender:
            sender_owned_territories.add(territory.upper()) # Abbreviations are case-insensitive
        elif owner.lower() == receiver:
            receiver_owned_territories.add(territory.upper())
            
    # Find all territories involved in sender's orders
    origin_territories = set(sender_units_orders.keys()) # e.g., {'Arm', 'Bul', 'Ank'}
    target_territories = set()
    for unit, order in sender_units_orders.items():
        if 'to' in order:
            target_territories.add(order['to'].upper()) # e.g., {'Sev', 'Rum'}

    # Create the new features (as 1s or 0s)
    new_features['mentions_own_territory'] = 0
    new_features['mentions_receiver_territory'] = 0
    new_features['mentions_origin_unit'] = 0
    new_features['mentions_target_territory'] = 0
    
    for abbr in mentioned_abbreviations:
        abbr_upper = abbr.upper()
        if abbr_upper in sender_owned_territories:
            new_features['mentions_own_territory'] = 1
        if abbr_upper in receiver_owned_territories:
            new_features['mentions_receiver_territory'] = 1
        if abbr_upper in origin_territories:
            new_features['mentions_origin_unit'] = 1
        if abbr_upper in target_territories:
            new_features['mentions_target_territory'] = 1
            
    return new_features

# --- 4. DATA LOADING ---

def load_and_flatten_data(filepath, game_state_lookup):
    X_text_list = []
    y_list = []
    X_new_features_list = [] 

    with jsonlines.open(filepath, 'r') as reader:
        for game in reader:
            game_id = game['game_id']
            years = game['years']
            seasons = game['seasons']
            speakers = game['speakers']
            
            for i in range(len(game['messages'])):
                sender_label = game['sender_labels'][i]
                if sender_label == "NOANNOTATION":
                    continue
                
                label = 1 if sender_label == False else 0
                message_text = game['messages'][i]
                year = years[i]
                season = seasons[i].lower() # <-- Use lowercase for lookup
                
                try:
                    current_game_state = game_state_lookup[game_id][year][season]
                except KeyError:
                    continue
                
                message_data = {
                    'text': message_text,
                    'speaker': speakers[i],
                    'receiver': game['receivers'][i],
                    'score_delta': int(game['game_score_delta'][i])
                }
                
                new_features = extract_features_from_state(message_data, current_game_state)
                
                X_text_list.append(message_text)
                y_list.append(label)
                X_new_features_list.append(new_features)

    print(f"Loaded and processed {len(X_text_list)} samples from {filepath}.")
    return X_text_list, X_new_features_list, np.array(y_list)

# --- 5. MODEL EVALUATION ---

def evaluate_model(model_name, y_true, y_pred):
    print(f"\n=============================================")
    print(f"PERFORMANCE TEST: {model_name}")
    print(f"=============================================")
    
    f1_lie_class = f1_score(y_true, y_pred, pos_label=1, average='binary')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    
    print(f"F1 Score (Lie Class): {f1_lie_class:.4f}")
    print(f"F1 Score (Macro Avg): {f1_macro:.4f}")

    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print("           Predicted 0 (Truth)  |  Predicted 1 (Lie)")
    print(f"Actual 0:    {cm[0][0]:<20} | {cm[0][1]:<20}")
    print(f"Actual 1:    {cm[1][0]:<20} | {cm[1][1]:<20}")
    
    print("\nFull Classification Report:")
    print(classification_report(y_true, y_pred, target_names=['Class 0 (Truth)', 'Class 1 (Lie)'], zero_division=0))

# --- 6. NEW: NEURAL NETWORK PIPELINE ---
if __name__ == '__main__':
    
    # --- Set Paths ---
    GAME_STATE_DIRECTORY = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "moves") 

    TRAIN_FILE = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "data", "train.jsonl") 
    VAL_FILE = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "data", "validation.jsonl") 
    TEST_FILE = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "data", "test.jsonl") 
    
    VOCAB_SIZE = 10000
    MAX_LEN = 120
    EMBEDDING_DIM = 64
    LSTM_UNITS = 64

    # --- Load Game State Dictionary ---
    game_state_lookup = create_game_state_dictionary(GAME_STATE_DIRECTORY)
    if not game_state_lookup:
        print("Game state lookup dictionary is empty. Exiting.")
        sys.exit(1)
        
    # --- Load all data ---
    print("Loading all datasets...")
    X_train_text, X_train_features_list, y_train = load_and_flatten_data(TRAIN_FILE, game_state_lookup)
    X_val_text, X_val_features_list, y_val = load_and_flatten_data(VAL_FILE, game_state_lookup)
    X_test_text, X_test_features_list, y_test = load_and_flatten_data(TEST_FILE, game_state_lookup)

    if len(X_train_text) == 0:
        print("CRITICAL ERROR: No training data was loaded.")
        sys.exit(1)

    # --- 1. PREPARE TEXT INPUT (Tokenizer + Padding) ---
    print("\nPreprocessing Text Data...")
    tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train_text) # Fit ONLY on training text
    
    X_train_seq = tokenizer.texts_to_sequences(X_train_text)
    X_val_seq = tokenizer.texts_to_sequences(X_val_text)
    X_test_seq = tokenizer.texts_to_sequences(X_test_text)
    
    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')
    
    print(f"Text data shape: {X_train_pad.shape}")

    # --- 2. PREPARE GAME STATE INPUT (DictVectorizer + Scaling) ---
    print("Preprocessing Game State Data...")
    feature_vectorizer = DictVectorizer(sparse=False)  # Use dense arrays instead of sparse
    X_train_features_numeric = feature_vectorizer.fit_transform(X_train_features_list)
    X_val_features_numeric = feature_vectorizer.transform(X_val_features_list)
    X_test_features_numeric = feature_vectorizer.transform(X_test_features_list)
    
    scaler = StandardScaler()  # Now works with dense arrays
    X_train_features_scaled = scaler.fit_transform(X_train_features_numeric)
    X_val_features_scaled = scaler.transform(X_val_features_numeric)
    X_test_features_scaled = scaler.transform(X_test_features_numeric)
    
    num_game_features = X_train_features_scaled.shape[1]
    print(f"Game state feature names: {feature_vectorizer.feature_names_}")
    print(f"Game state data shape: {X_train_features_scaled.shape}")

    # --- 3. DEFINE THE DUAL-INPUT MODEL ---
    print("\nBuilding Neural Network Model...")
    
    # Input 1: Text Sequences
    text_input_layer = Input(shape=(MAX_LEN,), name='text_input')
    text_model = Embedding(input_dim=VOCAB_SIZE+1, output_dim=EMBEDDING_DIM, input_length=MAX_LEN)(text_input_layer)
    text_model = SpatialDropout1D(0.2)(text_model) # Dropout for embeddings
    text_model = LSTM(LSTM_UNITS, dropout=0.2, recurrent_dropout=0.2, name='lstm_layer')(text_model)
    
    # Input 2: Game State Features
    game_state_input_layer = Input(shape=(num_game_features,), name='game_state_input')
    
    # Combine (Concatenate)
    combined = concatenate([text_model, game_state_input_layer], name='concatenate')
    
    # Classifier Head
    output = Dense(64, activation='relu', name='dense_1')(combined)
    output = Dropout(0.5, name='dropout')(output)
    output = Dense(1, activation='sigmoid', name='output_layer')(output)
    
    model = Model(inputs=[text_input_layer, game_state_input_layer], outputs=output)
    
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    model.summary()

    # --- 4. HANDLE CLASS IMBALANCE ---
    # This is CRITICAL for getting a good F1 score
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    class_weight_dict = dict(enumerate(class_weights))
    print(f"Class Weights (for balancing): {class_weight_dict}")

    # --- 5. TRAIN THE MODEL ---
    print("\nTraining the model...")
    
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=3,         # Stop after 3 epochs with no improvement
        restore_best_weights=True # Keep the best model, not the last
    )
    
    history = model.fit(
        [X_train_pad, X_train_features_scaled], # Pass inputs as a list
        y_train,
        epochs=15,
        batch_size=64,
        validation_data=([X_val_pad, X_val_features_scaled], y_val),
        callbacks=[early_stopping],
        class_weight=class_weight_dict # Apply the weights
    )

    # --- 6. EVALUATE THE FINAL MODEL ---
    print("\nEvaluating final model on test set...")
    y_pred_probs = model.predict([X_test_pad, X_test_features_scaled])
    evaluate_model("Dual-Input Neural Network", y_test, y_pred_probs)

Starting to build game state dictionary from: c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\moves
Successfully loaded and indexed 342 game state files.
Loading all datasets...
Loaded and processed 13132 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\train.jsonl.
Loaded and processed 1416 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\validation.jsonl.
Loaded and processed 2741 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\test.jsonl.

Preprocessing Text Data...
Loaded and processed 13132 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\train.jsonl.
Loaded and processed 1416 samples from c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\validation.jsonl.
Loaded and processed 2741 samples from c:\work environment\Projects\amazon-spam-review\dat

ValueError: Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.